In [1]:
import requests
from bs4 import BeautifulSoup
import os

import time
from datetime import datetime

# Function to download a file from a URL
def download_file(url, folder_path):
    local_filename = url.split('/')[-1]
    # Create folder if it doesn't exist
    os.makedirs(folder_path, exist_ok=True)
    # Full path for saving the file
    full_path = os.path.join(folder_path, local_filename)
    # Stream the download to handle large files
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(full_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    return full_path

# URL of the webpage containing the CSV links
webpage_url = 'https://data.everef.net/market-orders/'

last_checked_minute = datetime.now().minute

# Send a GET request to fetch the webpage content
response = requests.get(webpage_url)
response.raise_for_status()  # Check if the request was successful

# Parse the webpage content
soup = BeautifulSoup(response.content, 'html.parser')

# Find all links ending with .csv
csv_links = soup.find_all('a', href=True)
csv_urls = [link['href'] for link in csv_links if link['href'].endswith('.csv.bz2')]

# Folder path to save the downloaded CSV files
folder_path = 'data/eve_ref_market_orders/latest'

# Download each CSV file
for csv_url in csv_urls:
    full_csv_url = requests.compat.urljoin(webpage_url, csv_url)
    print(f'Downloading {full_csv_url}')
    download_file(full_csv_url, folder_path)

print('Download complete.')


Download complete.


In [2]:
import bz2
import os

def extract_bz2_files(folder_path):
    # List all files in the directory
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.bz2'):
            file_path = os.path.join(folder_path, file_name)
            output_file_path = os.path.splitext(file_path)[0]  # Remove .bz2 extension

            with bz2.BZ2File(file_path, 'rb') as file:
                with open(output_file_path, 'wb') as new_file:
                    for data in iter(lambda: file.read(100 * 1024), b''):
                        new_file.write(data)
                        
            
            print(f'Extracted {file_path} to {output_file_path}')

extract_bz2_files(folder_path)


Extracted data/eve_ref_market_orders/latest\market-orders-latest.v3.csv.bz2 to data/eve_ref_market_orders/latest\market-orders-latest.v3.csv


In [3]:
# delete all files with .bz2 extension:
for file_name in os.listdir(folder_path):
    if file_name.endswith('.bz2'):
        os.remove(os.path.join(folder_path, file_name))
        print(f'Deleted {file_name}')

Deleted market-orders-latest.v3.csv.bz2


In [1]:
from src import update_orderbook as uo

In [2]:
uo.update_book()

Download complete.
Extracted data/eve_ref_market_orders/latest\market-orders-latest.v3.csv.bz2 to data/eve_ref_market_orders/latest\market-orders-latest.v3.csv
Deleted data/eve_ref_market_orders/latest\market-orders-latest.v3.csv.bz2
